# SIMO Evaluation Playground

In [ ]:
# Import necessary libraries for PyTorch, optimization, file handling, and configuration
import torch
import torch.optim as optim
import os
import yaml
import wandb
import numpy as np
import random

from jinja2 import Environment, FileSystemLoader

# Import custom modules for dataset creation, network setup, MTL, training utils, and general utils
from training.create_dataset import *
from training.create_network import *
from models.dinov2.mtl.multitasker import MTLDinoV2
from training.utils import create_task_flags, TaskMetric, eval
from utils import torch_load, torch_save, get_data_loaders, initialize_wandb

# Login to wandb for experiment tracking
wandb.login()

## Configuration

Loading the experiment configuration from a Jinja2 template file. This includes hyperparameters for training, model merging settings, and dataset configurations. The configuration is rendered and parsed as YAML for easy access throughout the notebook.

In [ ]:
# Load and render the configuration template
env = Environment(loader=FileSystemLoader("."))
template = env.get_template("config/mtl.yaml.j2")
rendered_yaml = template.render()
config = yaml.safe_load(rendered_yaml)

In [ ]:
# Initialize Weights & Biases for logging
initialize_wandb(
    project=config["wandb"]["project"],
    group=f"{config['training_params']['network']}",
    job_type="task_specific",
    mode="offline",
    config={
        "task": config["training_params"]["task"],
        "network": config["training_params"]["network"],
        "dataset": config["training_params"]["dataset"],
        "epochs": config["training_params"]["total_epochs"],
        "batch_size": config["training_params"]["batch_size"],
        "seed": config["training_params"]["seed"],
    },
)

In [ ]:
# Set random seeds for reproducibility
torch.manual_seed(config["training_params"]["seed"])
np.random.seed(config["training_params"]["seed"])
random.seed(config["training_params"]["seed"])

# Determine the device for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Model Loading and Evaluation

Define the tasks for evaluation, load the data loaders, initialize the evaluation metric, load the pre-trained model, and perform evaluation on the test set.

In [ ]:
# Define the tasks for evaluation with their parameters
train_tasks = {
  "seg": {"num_classes": 13,},
  "depth": {"num_classes": 1, "min_depth": 0.00, "max_depth": 10.0,},
  "normal": {"num_classes": 3,}
}

# Load data loaders for training, validation, and testing
train_loader, val_loader, test_loader = get_data_loaders(config)

# Initialize the evaluation metric
test_metric = TaskMetric(
    train_tasks,
    train_tasks,
    config["training_params"]["batch_size"],
    1,
    config["training_params"]["dataset"],
)

# Load the pre-trained model
model = torch_load(
    "checkpoints/nyuv2/dinov2/dpt-add_small_head/mtl/final/simo_model.pt"
)
model.to(device)

# Perform evaluation on the test set
eval(1, model, test_loader, test_metric)